## UCBE HW11 Web Scraping
Feb 2018 A.Lam

Deliverable Variables:
- news_title
- news_p
- featured_image_url
- mars_weather
- table_string
- hemisphere_image_urls

In [1]:
# Dependencies 
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [2]:
urls = [
    'https://mars.nasa.gov/news/',
    'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars',
    'https://twitter.com/marswxreport?lang=en',
    'http://space-facts.com/mars/',
    'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
]

Scrape nasa.gov for latest news article(s)

In [3]:
browser = Browser("chrome",headless=False)
browser.visit(urls[0])
soup = bs(browser.html, 'html.parser')
# print(soup.prettify())

In [4]:
slides = soup.body.find_all('li',class_='slide')
articles = []
for slide in slides:
    title = slide.h3.text
    teaser = slide.find_all('div',class_='article_teaser_body')[0].text
    articles.append({'title' : title, 'description' : teaser})

In [5]:
news_title = articles[0]['title']
news_p = articles[0]['description']
print(f'{news_title}:\n{news_p}')

Seven Ways Mars InSight is Different:
NASA has a long and successful track record at Mars. Since 1965, it has flown by, orbited, landed and roved across the surface of the Red Planet. What can InSight -- planned for launch in May -- do that hasn’t been done before?


JPL Mars Space Images - Featured Image
Visit the url for JPL's Featured Space Image here.

Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

Make sure to find the image url to the full size .jpg image.

Make sure to save a complete url string for this image.

In [6]:
browser.visit(urls[1])
soup = bs(browser.html, 'html.parser')
# print(soup.prettify())

In [7]:
JPLURL = 'https://www.jpl.nasa.gov'
slides = soup.find_all('li',class_='slide')
featured_images = []
for slide in slides:
    try:
        featured_images.append(JPLURL + slide.a['data-fancybox-href'])
    except KeyError:
        pass

In [8]:
featured_image_url = featured_images[0]
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA22222_hires.jpg


Mars Weather

In [9]:
browser.visit(urls[2])
soup = bs(browser.html, 'html.parser')
# print(soup.prettify())

In [10]:
mars_username = '@MarsWxReport'
tweets = soup.body.find_all('div',class_='tweet')
mars_posts = []
for tweet in tweets:
    if tweet.find_all('span',class_='username')[0].text == mars_username:
        mars_posts.append(tweet.find_all('p',class_='tweet-text')[0].text)

In [11]:
mars_weather = mars_posts[0]
print(mars_weather)

Sol 1970 (Feb 20, 2018), Sunny, high -16C/3F, low -77C/-106F, pressure at 7.32 hPa, daylight 05:38-17:26


Mars Facts

In [12]:
browser.visit(urls[3])
mars_facts_df = pd.read_html(browser.html)[0]
table_string = mars_facts_df.to_html()
print(table_string)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>Recorde

In [13]:
browser.visit(urls[4])
browser.click_link_by_partial_text('Enhanced')


In [14]:
browser.click_link_by_partial_text('Back')

In [32]:
hemisphere_links = browser.find_link_by_partial_text('Hemisphere')
link_text = []
for link in hemisphere_links:
    link_text.append(link.text)
hemisphere_image_urls = []
for link in link_text:
    browser.click_link_by_partial_text(link)
    hemisphere_image_urls.append({
        'title' : link[:-9],
        'tif_url' : browser.find_link_by_partial_text('Original')['href'],
        'jpg_url' : browser.find_link_by_text('Sample')['href'],
    })
    browser.click_link_by_partial_text('Back')
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere', 'tif_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif', 'jpg_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'tif_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif', 'jpg_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'tif_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif', 'jpg_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'tif_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif', 'jpg_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
